<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


## Final Project - Build an ML Pipeline for Airfoil noise prediction


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model 
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise
 
 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


Before you start attempting this project it is highly recommended that you finish the practice project.


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [72]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [73]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [74]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col


### Task 2 - Create a spark session


In [75]:
spark = SparkSession.builder.appName("AirfoilSelfNoisePrediction").getOrCreate()


### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [76]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2023-11-08 21:31:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv.1’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.001s  

2023-11-08 21:31:22 (40.7 MB/s) - ‘NASA_airfoil_noise_raw.csv.1’ saved [60682/60682]



Load the dataset into the spark dataframe


In [77]:
df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True)


### Task 4 - Print top 5 rows of the dataset


In [78]:
df.show(5)


+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|            0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|            0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|            0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|            0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|            0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



### Task 6 - Print the total number of rows in the dataset


In [79]:
rowcount1 = df.count()
print(rowcount1)


1522


### Task 7 - Drop all the duplicate rows from the dataset


In [80]:
df = df.dropDuplicates()


### Task 8 - Print the total number of rows in the dataset


In [81]:
rowcount2 = df.count()
print("Total number of rows after dropping duplicates:", rowcount2)


[Stage 82:===============================================>     (181 + 10) / 200]

Total number of rows after dropping duplicates: 1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [82]:
df = df.dropna()


### Task 10 - Print the total number of rows in the dataset


In [83]:
rowcount3 = df.count()
print("Total number of rows after dropping rows with null values:", rowcount3)


[Stage 85:====================================================> (193 + 7) / 200]

Total number of rows after dropping rows with null values: 1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"Drop


In [84]:
df = df.withColumnRenamed("SoundLevel", "SoundLevelDecibels")


### Task 12 - Save the dataframe in parquet formant, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [86]:
df.write.parquet("NASA_airfoil_noise_cleaned.parquet")


#### Part 1 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [87]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [89]:
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")


In [97]:
# List of columns to be converted to numeric types
string_cols = ['Frequency', 'AngleOfAttack', 'ChordLength', 'FreeStreamVelocity', 'SuctionSideDisplacement', 'SoundLevelDecibels']

# Convert string columns to numeric types
for col_name in string_cols:
    df = df.withColumn(col_name, col(col_name).cast("float"))

### Task 2 - Print the total number of rows in the dataset


In [98]:
rowcount4 = df.count()
print(rowcount4)


[Stage 92:====================================>                     (5 + 3) / 8]

1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [99]:
input_cols = ['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement']
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")


### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [100]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")


### Task 5 - Define the StandardScaler pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"

**Note:You need to use the scaledfeatures retreived in the previous step.**


In [101]:
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")


### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [102]:
pipeline = Pipeline(stages=[assembler, scaler, lr])


### Task 7 - Split the data


In [103]:
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)


### Task 8 - Fit the pipeline


In [104]:
pipelineModel = pipeline.fit(trainingData)


23/11/08 21:37:03 WARN util.Instrumentation: [8a89d1f1] regParam is zero, which might cause numerical instability and overfitting.


#### Part 2 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [105]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [106]:
predictions = pipelineModel.transform(testingData)


### Task 2 - Print the MSE


In [107]:
evaluator_mse = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mse")

# Calculate the MSE
mse = evaluator_mse.evaluate(predictions)
print("Mean Squared Error (MSE):", mse)


[Stage 104:===================================>                     (5 + 3) / 8]

Mean Squared Error (MSE): 24.185951085908513


### Task 3 - Print the MAE


In [108]:
evaluator_mae = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="mae")

# Calculate the MAE
mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error (MAE):", mae)


[Stage 106:=================================================>       (7 + 1) / 8]

Mean Absolute Error (MAE): 3.855167546755095


### Task 4 - Print the R-Squared(R2)


In [109]:
evaluator_r2 = RegressionEvaluator(labelCol="SoundLevelDecibels", predictionCol="prediction", metricName="r2")

# Calculate the R2 score
r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2):", r2)


[Stage 108:===================================>                     (5 + 3) / 8]

R-squared (R2): 0.5092051559332174


#### Part 3 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [110]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  24.19
Mean Absolute Error =  3.86
R Squared =  0.51
Intercept =  132.93


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [111]:
pipelineModel.write().overwrite().save("Final_Project")


### Task 2 - Load the model from the path "Final_Project"


In [112]:
loadedPipelineModel = pipelineModel.load("Final_Project")


### Task 3 - Make predictions using the loaded model on the testdata


In [113]:
predictions = loadedPipelineModel.transform(testingData)


### Task 4 - Show the predictions


In [114]:
predictions.select("SoundLevelDecibels", "prediction").show(5)


[Stage 130:>                                                        (0 + 1) / 1]

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           118.129|125.31263566756155|
|           127.315|123.63648783931461|
|           123.595|127.53368605246162|
|           119.698|131.60885425349448|
|           121.055|127.90076502516814|
+------------------+------------------+
only showing top 5 rows



#### Part 4 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [115]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.974
Coefficient for AngleOfAttack is -2.3077
Coefficient for ChordLength is -3.2911
Coefficient for FreeStreamVelocity is 1.4681
Coefficient for SuctionSideDisplacement is -2.0418


### Stop Spark Session


In [ ]:
spark.stop()

## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
